In [1]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/'

fileNames = ["parrot.bmp", "clock.bmp", "chessboard.bmp", "bart.png", "firetruck.jpg"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)


parrot = cv2.imread('parrot.bmp')           # Read image
parrot = cv2.cvtColor(parrot, cv2.COLOR_BGR2GRAY) # Convert to RGB

In [12]:
# frame: input frame from CMOS camera
# beta:  treshold on potential hit detection
# gamma: treshold on averaged Frame pixels value sum
# delta: treshold on fraction of black piels count

def alg1(frames: list[np.array], resolution: tuple[int ,int], beta=120, gamma=40, delta=0.04):
    potentialHits = []
    (Y, X) = resolution
    while len(frames) > 0:
        frame = frames.pop(0)
        r,g,b = cv2.split(frame)
        frameSum = r + g + b
        maxFrameSum = np.max(framSum)
        zeroCountFrameSum = 0
        for j in range(Y):
            for i in range(X):
                zeroCountFrameSum += 1 if maxFrameSum[j, i] == 0 else 0
        sumFrameSum = frameSum.sum()
        avgFrameSum = sumFrameSum / (Y*X)
        blacksFrameSum = zeroCountFrameSum / (Y*X)
        if maxFrameSum > beta and avgFrameSum < gamma and blacksFrameSum > delta:
            potentialHits.append(frame)
    return potentialHits

In [15]:
def alg2(frames: list[np.array], resolution: tuple[int ,int], zoneSize):
    potentialHits = []
    triggerMapWidth = resolution[1] // zoneSize
    triggerMapHeight = resolution[0] // zoneSize
    heat = np.zeros(resolution)
    pixelScores = np.zeros(resolution)
    treshold = 0
    while len(frames) > 0:
        frame = frames.pop(0)
        r,g,b = cv2.split(frame)
        total = r + g + b

        diff = total - heat

        for b in range(resolution[0]):
            for a in range(resolution[1]):
                if total[b, a] > 4 and diff[b, a] > 4 * heat[b, a]:
                    pixelScores[b, a] = diff[b, a] * diff[b, a]
                else:
                    pixelScores[b, a] = 0
                heat[b, a] = 0.03 * total[b, a] + (1 - 0.03) * heat[b, a]

        blockScores = np.zeros((triggerMapHeight, triggerMapWidth))
        for b in range(resolution[0]):
            for a in range(resolution[1]):
                blockScores[b, a] += pixelScores[b // zoneSize, a // zoneSize]

        un = np.sort(blockScores, axis=None)
        max2ND = un[len(un) - 2]
        
        if treshold == 0:
            un = np.sort(blockScores, axis=None)
            treshold = max2ND
        else:
            treshold = 0.1 * max2ND + (1 - 0.1) * treshold

        for b in range(1, triggerMapHeight - 1):
            for a in range(1, triggerMapWidth - 1):
                if blockScores[b, a] > 2 * treshold:
                    potentialHits.append(frame)
    return potentialHits

In [17]:
def crop(frame, edge):
    return

def max_pooling(frame, num):
    return

def gaussian_blur(frame, kernel):
    return


# edge:       width of image edge in pixel which will be cropped during processing
# kernel:     gaussian kernel matrix
# imc:        number of initial iterations devoted to initialise moving average
# resolution: CMOS image resoultion
# alfa:       moving average memory
# theta:      treshold of potential hit detections

def alg3(frames: list[np.array], resolution: tuple[int ,int], edge, kernel, imc, alfa, theta):
    potentialHits = []
    frameAVG = None
    while len(frames) > 0:
        frame = frames.pop(0)
        r,g,b = cv2.split(frame)
        frameSum = r + g + b
        frameCrop = crop(frameSum, edge) # TODO: crop()
        framePool = max_pooling(frameCrop, 2) # TODO: max_pooling()
        frameGaussian = gaussian_blur(framePool, kernel) # TODO: gaussian_blur()

        if imc <= 0:
            for b in range(resolution[0]):
                for a in range(resolution[1]):
                    if frameAVG[b, a] * 4 < frameGaussian[b, a] and framePool[b, a] > theta:
                        potentialHits.append(frame)
        else:
            imc -= 1

        if frameAVG is None:
            frameAVG = frameGaussian
        else:
            frameAVG = (alfa - 1) * frameAVG + alfa * frameGaussian
    return potentialHits

array([[0., 0.],
       [0., 0.],
       [0., 0.]])